# Streaming: Inferencia de Live Win Probability en Tiempo Real

**Objetivo:** Aplicar el modelo de ML entrenado (LWP) al stream de eventos en vivo para predecir probabilidades de victoria en tiempo real.

**Entrada:** Stream de eventos de Kafka

**Salida:** Predicciones continuas de:
- P(Victoria Local)
- P(Empate)
- P(Victoria Visitante)

**Arquitectura:** Spark Structured Streaming + RAPIDS GPU + ML Model

## 1. Setup y Configuración

In [ ]:
import os
import sys
import warnings
warnings.filterwarnings('ignore')

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.classification import RandomForestClassificationModel
from pyspark.ml.feature import VectorAssembler
from datetime import datetime

print(f"Setup complete - {datetime.now()}")

## 2. Inicializar Spark Session con GPU

In [ ]:
# Initialize Spark with RAPIDS GPU acceleration
spark = SparkSession.builder \
    .appName("StatsBomb-Streaming-ML-Inference-GPU") \
    .master("spark://spark-master:7077") \
    .config("spark.rapids.sql.enabled", "true") \
    .config("spark.plugins", "com.nvidia.spark.SQLPlugin") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.1") \
    .config("spark.sql.streaming.checkpointLocation", "/tmp/checkpoint-ml") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")

print(f"✓ Spark Version: {spark.version}")
print(f"✓ Spark Master: {spark.sparkContext.master}")
print(f"✓ Spark UI: http://localhost:4040")
print("\n📊 Monitor Spark UI para capturar métricas de ML en streaming")

## 3. Cargar Modelo Entrenado

In [ ]:
# Load the trained model
MODEL_PATH = "/work/models/lwp_model"

print(f"Cargando modelo desde {MODEL_PATH}...")

try:
    lwp_model = RandomForestClassificationModel.load(MODEL_PATH)
    print("✓ Modelo cargado exitosamente")
    print(f"✓ Número de árboles: {lwp_model.getNumTrees}")
    print(f"✓ Número de features: {lwp_model.numFeatures}")
except Exception as e:
    print(f"❌ Error cargando modelo: {e}")
    print("\n⚠️  Asegúrate de ejecutar primero el notebook 02_Entrenamiento_Modelo_LWP.ipynb")
    raise

## 4. Definir Schema y Conectar a Kafka

In [ ]:
# Define schema for StatsBomb events
event_schema = StructType([
    StructField("event", StructType([
        StructField("id", StringType(), True),
        StructField("index", IntegerType(), True),
        StructField("period", IntegerType(), True),
        StructField("timestamp", StringType(), True),
        StructField("minute", IntegerType(), True),
        StructField("second", IntegerType(), True),
        StructField("type", StructType([
            StructField("id", IntegerType(), True),
            StructField("name", StringType(), True)
        ]), True),
        StructField("team", StructType([
            StructField("id", IntegerType(), True),
            StructField("name", StringType(), True)
        ]), True),
        StructField("player", StructType([
            StructField("id", IntegerType(), True),
            StructField("name", StringType(), True)
        ]), True),
        StructField("location", ArrayType(DoubleType()), True),
        StructField("pass_end_location", ArrayType(DoubleType()), True),
    ]), True),
    StructField("metadata", StructType([
        StructField("producer_timestamp", StringType(), True),
        StructField("producer_id", StringType(), True)
    ]), True)
])

# Kafka configuration
KAFKA_BOOTSTRAP_SERVERS = "kafka:9092"
KAFKA_TOPIC = "statsbomb-360-events"

# Read from Kafka
kafka_stream = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", KAFKA_BOOTSTRAP_SERVERS) \
    .option("subscribe", KAFKA_TOPIC) \
    .option("startingOffsets", "latest") \
    .option("maxOffsetsPerTrigger", 10000) \
    .load()

# Parse JSON
parsed_stream = kafka_stream.select(
    col("timestamp").alias("kafka_timestamp"),
    from_json(col("value").cast("string"), event_schema).alias("data")
).select(
    "kafka_timestamp",
    "data.event.*",
    "data.metadata.*"
)

print("✓ Conectado a Kafka")
print(f"✓ Topic: {KAFKA_TOPIC}")

## 5. Feature Engineering para el Modelo

In [ ]:
# Extract and aggregate features needed for the model
# We need to maintain state of the match to calculate features

# First, extract basic features from events
events_with_features = parsed_stream \
    .withColumn("event_time", col("kafka_timestamp")) \
    .withColumn("event_type", col("type.name")) \
    .withColumn("team_id", col("team.id")) \
    .withColumn("team_name", col("team.name")) \
    .withColumn("is_pass", when(col("event_type") == "Pass", 1).otherwise(0)) \
    .withColumn("is_shot", when(col("event_type") == "Shot", 1).otherwise(0)) \
    .withColumn("is_goal", when((col("event_type") == "Shot"), 1).otherwise(0))  # Simplified

print("✓ Features básicos extraídos")

In [ ]:
# Aggregate features over 2-minute windows to create match state snapshots
# This simulates the current state of the match at each point in time

match_state = events_with_features \
    .withWatermark("event_time", "30 seconds") \
    .groupBy(
        window(col("event_time"), "2 minutes", "1 minute"),  # Sliding window
        col("team_name")
    ) \
    .agg(
        max("minute").alias("current_minute"),
        sum("is_goal").alias("goals"),
        sum("is_shot").alias("shots"),
        sum("is_pass").alias("passes"),
        count("*").alias("events")
    )

print("✓ Estado del partido agregado por ventanas")

In [ ]:
# Pivot to get home vs away team stats in the same row
# This is a simplified version - in production, you'd need to properly identify home/away teams

# For demonstration, we'll create synthetic match state features
# In a real scenario, you'd maintain global state or use more sophisticated streaming joins

features_for_model = match_state \
    .withColumn("window_start", col("window.start")) \
    .withColumn("window_end", col("window.end")) \
    .drop("window") \
    .withColumn(
        # Create a simplified match state
        "minute", col("current_minute")
    ) \
    .withColumn("home_score", col("goals")) \
    .withColumn("away_score", lit(0)) \
    .withColumn("score_diff", col("goals")) \
    .withColumn("home_shots", col("shots")) \
    .withColumn("away_shots", lit(0)) \
    .withColumn("shots_diff", col("shots")) \
    .withColumn("home_passes", col("passes")) \
    .withColumn("away_passes", lit(0)) \
    .withColumn("passes_diff", col("passes")) \
    .withColumn("home_possession", lit(0.5)) \
    .withColumn("time_remaining", lit(90) - col("minute"))

print("✓ Features para el modelo preparados")

## 6. Aplicar Modelo para Inferencia

In [ ]:
# Define the same feature columns used during training
feature_cols = [
    'minute', 'home_score', 'away_score', 'score_diff',
    'home_shots', 'away_shots', 'shots_diff',
    'home_passes', 'away_passes', 'passes_diff',
    'home_possession', 'time_remaining'
]

# Create vector assembler
assembler = VectorAssembler(
    inputCols=feature_cols,
    outputCol="features"
)

# Assemble features
data_with_features = assembler.transform(features_for_model)

# Apply model to streaming data
predictions = lwp_model.transform(data_with_features)

print("✓ Modelo aplicado al stream")
print("✓ Predicciones continuas generándose")

In [ ]:
# Extract probabilities for better display
predictions_formatted = predictions.select(
    col("window_start"),
    col("window_end"),
    col("team_name"),
    col("minute"),
    col("score_diff"),
    col("shots_diff"),
    col("passes_diff"),
    col("prediction"),
    col("probability"),
    # Extract individual probabilities
    col("probability").getItem(0).alias("prob_class_0"),
    col("probability").getItem(1).alias("prob_class_1"),
    col("probability").getItem(2).alias("prob_class_2")
) \
.withColumn(
    # Format probabilities as percentages
    "prediction_text",
    when(col("prediction") == 0.0, "HOME WIN")
    .when(col("prediction") == 1.0, "DRAW")
    .otherwise("AWAY WIN")
) \
.select(
    "window_start",
    "team_name",
    "minute",
    "score_diff",
    "shots_diff",
    "passes_diff",
    "prediction_text",
    (col("prob_class_0") * 100).alias("P_HOME_WIN_%"),
    (col("prob_class_1") * 100).alias("P_DRAW_%"),
    (col("prob_class_2") * 100).alias("P_AWAY_WIN_%")
) \
.orderBy("window_start")

print("✓ Predicciones formateadas para visualización")

## 7. Iniciar Stream - Output de Predicciones

In [ ]:
# Start streaming query with console output
query = predictions_formatted.writeStream \
    .outputMode("complete") \
    .format("console") \
    .option("truncate", "false") \
    .option("numRows", 30) \
    .trigger(processingTime="10 seconds") \
    .start()

print("="*80)
print("STREAMING ML INFERENCE ACTIVO")
print("="*80)
print(f"Query ID: {query.id}")
print(f"Status: {query.status}")
print("\n🤖 PREDICCIONES EN TIEMPO REAL:")
print("   - P_HOME_WIN_%: Probabilidad de victoria local")
print("   - P_DRAW_%: Probabilidad de empate")
print("   - P_AWAY_WIN_%: Probabilidad de victoria visitante")
print("\n📊 CAPTURA DE MÉTRICAS:")
print("   1. Abre Spark UI en http://localhost:4040")
print("   2. Compara métricas con el notebook 03 (sin ML)")
print("   3. Observa el overhead de inferencia ML:")
print("      - Batch Duration aumentado")
print("      - CPU/GPU utilization")
print("      - Memory usage")
print("="*80)
print("\n⚠️  Presiona el botón STOP para detener el streaming")
print("="*80)

In [ ]:
# Monitor query status
import time

try:
    while query.isActive:
        print(f"\n[{datetime.now().strftime('%H:%M:%S')}] ML Inference Query Status")
        print(f"Status: {query.status}")
        
        if query.recentProgress:
            latest = query.recentProgress[-1]
            print(f"Recent Progress:")
            print(f"  - Batch ID: {latest.get('batchId', 'N/A')}")
            print(f"  - Input Rows: {latest.get('numInputRows', 'N/A')}")
            print(f"  - Process Rate: {latest.get('processedRowsPerSecond', 'N/A'):.2f} rows/sec" if latest.get('processedRowsPerSecond') else "  - Process Rate: N/A")
            print(f"  - Duration: {latest.get('durationMs', {}).get('triggerExecution', 'N/A')} ms")
            print(f"  - ML Inference Overhead: observe en Spark UI")
        
        time.sleep(10)
        
except KeyboardInterrupt:
    print("\n⚠️  Deteniendo streaming...")
    query.stop()
    print("✓ Streaming detenido")

## 8. Detener Stream

In [ ]:
# Stop the streaming query
if query.isActive:
    print("Deteniendo streaming query...")
    query.stop()
    query.awaitTermination(timeout=30)
    print("✓ Query detenido")
else:
    print("Query no está activo")

## 9. Análisis de Rendimiento y Comparación

In [ ]:
print("="*80)
print("GUÍA DE ANÁLISIS DE RENDIMIENTO")
print("="*80)
print("\n1. MÉTRICAS CLAVE A CAPTURAR:")
print("   📊 Streaming Tab (Spark UI):")
print("      - Input Rate vs Process Rate")
print("      - Batch Duration (con vs sin ML)")
print("      - Scheduling Delay")
print("\n   ⚙️  Jobs Tab:")
print("      - Job Duration (comparar notebooks 03 vs 04)")
print("      - Shuffle metrics")
print("      - Task metrics")
print("\n   💾 Executors Tab:")
print("      - Memory utilization")
print("      - GPU utilization (si disponible)")
print("      - GC time")
print("\n2. COMPARACIÓN ARQUITECTURAS:")
print("   🔄 Para cambiar a Arquitectura 2 (CPU):")
print("      1. Editar spark-conf/spark-defaults.conf")
print("      2. Comentar líneas:")
print("         # spark.plugins=com.nvidia.spark.SQLPlugin")
print("         # spark.rapids.sql.enabled=true")
print("      3. Editar docker-compose.yml")
print("      4. Remover secciones 'deploy.resources.reservations.devices'")
print("      5. Reiniciar servicios: docker-compose down && docker-compose up -d")
print("      6. Re-ejecutar notebooks")
print("\n3. MÉTRICAS A COMPARAR (GPU vs CPU):")
print("   ✓ Throughput (eventos/segundo)")
print("   ✓ Latencia promedio por batch")
print("   ✓ Job time total")
print("   ✓ Shuffle read/write")
print("   ✓ Memory spill")
print("   ✓ GC time")
print("   ✓ CPU/GPU utilization")
print("="*80)
print("\n4. EXPORTAR MÉTRICAS:")
print("   - Spark UI tiene opción de exportar JSON")
print("   - Captura screenshots de gráficas clave")
print("   - Usa las métricas de query.recentProgress para análisis")
print("="*80)

In [ ]:
# Export query metrics if available
if hasattr(query, 'recentProgress') and query.recentProgress:
    print("\nÚltimas métricas capturadas:")
    import json
    
    for i, progress in enumerate(query.recentProgress[-3:], 1):
        print(f"\n--- Batch {progress.get('batchId', 'N/A')} ---")
        print(json.dumps(progress, indent=2))
else:
    print("\nNo hay métricas disponibles. Ejecuta el query primero.")

In [ ]:
# Clean up
# spark.stop()
print("\n✓ Notebook completado")
print("\nNota: Spark session sigue activa.")
print("Ejecuta 'spark.stop()' cuando termines todas las pruebas.")